In [2]:
import pandas as pd

In [3]:
!pip install pandas numpy matplotlib tensorflow scikit-learn


In [4]:
!pip install pandas numpy matplotlib tensorflow scikit-learn


*italicised text*# New section

In [5]:

import pandas as pd



FileNotFoundError: [Errno 2] No such file or directory: 'train.csv'

In [13]:
# Load datasets
train_df = pd.read_csv("/train.csv")
store_df = pd.read_csv("/store.csv")

# Check first few rows
print("Train Data Overview:")
print(train_df.head())

print("\nStore Data Overview:")
print(store_df.head())

# Check dataset info
print("\nTrain Data Info:")
print(train_df.info())

print("\nStore Data Info:")
print(store_df.info())

Train Data Overview:
   Store  DayOfWeek        Date    Sales  Customers  Open  Promo StateHoliday  \
0      1          5  2015-07-31   5263.0      555.0   1.0    1.0            0   
1      2          5  2015-07-31   6064.0      625.0   1.0    1.0            0   
2      3          5  2015-07-31   8314.0      821.0   1.0    1.0            0   
3      4          5  2015-07-31  13995.0     1498.0   1.0    1.0            0   
4      5          5  2015-07-31   4822.0      559.0   1.0    1.0            0   

   SchoolHoliday  
0            1.0  
1            1.0  
2            1.0  
3            1.0  
4            1.0  

Store Data Overview:
   Store StoreType Assortment  CompetitionDistance  CompetitionOpenSinceMonth  \
0      1         c          a               1270.0                        9.0   
1      2         a          a                570.0                       11.0   
2      3         a          a              14130.0                       12.0   
3      4         c          c   

🔹 Step 3: Preprocess Data for AI Forecasting

In [14]:
import numpy as np

# Merge train data with store details using "Store" column
df = pd.merge(train_df, store_df, on="Store", how="left")

# Convert date column to datetime format
df["Date"] = pd.to_datetime(df["Date"])
df.sort_values(by="Date", inplace=True)

# Handle missing values
df.fillna(method="ffill", inplace=True)  # Forward fill missing values

# Create new features for better AI forecasting
df["Year"] = df["Date"].dt.year
df["Month"] = df["Date"].dt.month
df["Week"] = df["Date"].dt.isocalendar().week  # Extract correct week
df["DayOfWeek"] = df["Date"].dt.dayofweek

print("\n✅ Data Preprocessing Completed! Ready for AI model training.")


ValueError: time data "2015-04-" doesn't match format "%Y-%m-%d", at position 101. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [15]:
# Identify rows with problematic dates
print("Checking for invalid date formats:")
print(df[df["Date"].str.len() < 10])  # Shows rows with incomplete dates

# Remove invalid dates or fill missing day values
df["Date"] = df["Date"].apply(lambda x: x + "-01" if len(x) < 10 else x)  # Fill missing days with "01"

# Convert to proper datetime format
df["Date"] = pd.to_datetime(df["Date"], format="%Y-%m-%d", errors="coerce")  # Coerce forces conversion

# Drop any remaining NaN dates
df = df.dropna(subset=["Date"])

print("\n✅ Date column cleaned and converted successfully!")

Checking for invalid date formats:
        Store  DayOfWeek      Date  Sales  Customers  Open  Promo  \
112103    604          3  2015-04-    NaN        NaN   NaN    NaN   

       StateHoliday  SchoolHoliday StoreType Assortment  CompetitionDistance  \
112103          NaN            NaN         d          a              10950.0   

        CompetitionOpenSinceMonth  CompetitionOpenSinceYear  Promo2  \
112103                        3.0                    2014.0       0   

        Promo2SinceWeek  Promo2SinceYear PromoInterval  
112103              NaN              NaN           NaN  

✅ Date column cleaned and converted successfully!


In [17]:
# Drop rows where Date conversion failed
df = df.dropna(subset=["Date"])

# Handle missing values using forward fill
df.ffill(inplace=True)  # Correct method instead of deprecated fillna()

# Create new features for better AI forecasting
df["Year"] = df["Date"].dt.year
df["Month"] = df["Date"].dt.month
df["Week"] = df["Date"].dt.isocalendar().week  # Extract week correctly
df["DayOfWeek"] = df["Date"].dt.dayofweek

print("\n✅ Dataset merged, Date fixed, and missing values handled! Ready for AI model training.")

AttributeError: Can only use .dt accessor with datetimelike values

🔹 Step 4: Train AI Model for Demand Forecasting

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112104 entries, 0 to 112103
Data columns (total 18 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   Store                      112104 non-null  int64  
 1   DayOfWeek                  112104 non-null  int64  
 2   Date                       112104 non-null  object 
 3   Sales                      112104 non-null  float64
 4   Customers                  112104 non-null  float64
 5   Open                       112104 non-null  float64
 6   Promo                      112104 non-null  float64
 7   StateHoliday               112104 non-null  object 
 8   SchoolHoliday              112104 non-null  float64
 9   StoreType                  112104 non-null  object 
 10  Assortment                 112104 non-null  object 
 11  CompetitionDistance        112104 non-null  float64
 12  CompetitionOpenSinceMonth  112104 non-null  float64
 13  CompetitionOpenSinceYear   11

In [19]:
# Ensure Date column is in proper datetime format
df["Date"] = pd.to_datetime(df["Date"], errors="coerce")

# Drop any remaining NaN values in Date
df.dropna(subset=["Date"], inplace=True)

# Extract time-based features
df["Year"] = df["Date"].dt.year
df["Month"] = df["Date"].dt.month
df["Week"] = df["Date"].dt.isocalendar().week  # Correct method for extracting week number
df["DayOfWeek"] = df["Date"].dt.dayofweek

print("\n✅ Date conversion completed & features extracted!")



✅ Date conversion completed & features extracted!


In [20]:
print("Missing Values Summary:")
print(df.isnull().sum())


Missing Values Summary:
Store                        0
DayOfWeek                    0
Date                         0
Sales                        0
Customers                    0
Open                         0
Promo                        0
StateHoliday                 0
SchoolHoliday                0
StoreType                    0
Assortment                   0
CompetitionDistance          0
CompetitionOpenSinceMonth    0
CompetitionOpenSinceYear     0
Promo2                       0
Promo2SinceWeek              1
Promo2SinceYear              1
PromoInterval                1
Year                         0
Month                        0
Week                         0
dtype: int64


In [21]:
print("\nDataset Info:")
print(df.info())



Dataset Info:
<class 'pandas.core.frame.DataFrame'>
Index: 112103 entries, 0 to 112102
Data columns (total 21 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   Store                      112103 non-null  int64         
 1   DayOfWeek                  112103 non-null  int32         
 2   Date                       112103 non-null  datetime64[ns]
 3   Sales                      112103 non-null  float64       
 4   Customers                  112103 non-null  float64       
 5   Open                       112103 non-null  float64       
 6   Promo                      112103 non-null  float64       
 7   StateHoliday               112103 non-null  object        
 8   SchoolHoliday              112103 non-null  float64       
 9   StoreType                  112103 non-null  object        
 10  Assortment                 112103 non-null  object        
 11  CompetitionDistance        112103 non-null

🔹 Step 5: Train the AI Forecasting Model

In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.model_selection import train_test_split
import numpy as np

# Select relevant features for forecasting
df_forecast = df[["Sales", "Month", "DayOfWeek", "Store"]]  # Using numeric features
X = df_forecast.iloc[:-1].values  # Past sales data
y = df_forecast["Sales"].iloc[1:].values.reshape(-1, 1)  # Future sales predictions

# Split into training & testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Reshape data for LSTM model
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Build AI model (LSTM for time-series forecasting)
model = Sequential([
    LSTM(50, activation='relu', return_sequences=True, input_shape=(X_train.shape[1], 1)),
    Dense(1)
])

# Compile & train model
model.compile(optimizer='adam', loss='mse')
model.fit(X_train, y_train, epochs=50, batch_size=16)

# Predict future demand
y_pred = model.predict(X_test)

print("✅ AI Forecasting Model Trained Successfully!")


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
5606/5606 ━━━━━━━━━━━━━━━━━━━━ 27s 4ms/step - loss: 16015695.0000
Epoch 2/50
5606/5606 ━━━━━━━━━━━━━━━━━━━━ 23s 4ms/step - loss: 12356414.0000
Epoch 3/50
5606/5606 ━━━━━━━━━━━━━━━━━━━━ 24s 4ms/step - loss: 12158260.0000
Epoch 4/50
5606/5606 ━━━━━━━━━━━━━━━━━━━━ 23s 4ms/step - loss: 12289684.0000
Epoch 5/50
5606/5606 ━━━━━━━━━━━━━━━━━━━━ 24s 4ms/step - loss: 11969998.0000
Epoch 6/50
5606/5606 ━━━━━━━━━━━━━━━━━━━━ 41s 4ms/step - loss: 11974647.0000
Epoch 7/50
5606/5606 ━━━━━━━━━━━━━━━━━━━━ 40s 4ms/step - loss: 12215538.0000
Epoch 8/50
5606/5606 ━━━━━━━━━━━━━━━━━━━━ 41s 4ms/step - loss: 12054669.0000
Epoch 9/50
5606/5606 ━━━━━━━━━━━━━━━━━━━━ 41s 4ms/step - loss: 12192139.0000
Epoch 10/50
5606/5606 ━━━━━━━━━━━━━━━━━━━━ 27s 5ms/step - loss: 12053268.0000
Epoch 11/50
5606/5606 ━━━━━━━━━━━━━━━━━━━━ 44s 5ms/step - loss: 14146878.0000
Epoch 12/50
5606/5606 ━━━━━━━━━━━━━━━━━━━━ 30s 5ms/step - loss: 12035780.0000
Epoch 13/50
5606/5606 ━━━━━━━━━━━━━━━━━━━━ 29s 5ms/step - loss: 11778414.

🔹 Step 6: Evaluate Model Accuracy

In [26]:
# Ensure y_pred has the same shape as y_test
y_pred = y_pred[:len(y_test)]  # Trim predictions to match test samples

# Re-run evaluation
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print(f"\n✅ Model Evaluation Completed!")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Mean Squared Error (MSE): {mse:.2f}")



✅ Model Evaluation Completed!
Mean Absolute Error (MAE): 3716.25
Mean Squared Error (MSE): 23139159.41


🔹 Step 8: Save & Upload the Model

In [27]:
import joblib

# Save trained AI model
joblib.dump(model, "retail_demand_forecaster.pkl")

# Save predictions to CSV file
import pandas as pd
predictions_df = pd.DataFrame({"Actual Sales": y_test, "Predicted Sales": y_pred})
predictions_df.to_csv("sales_predictions.csv", index=False)

print("✅ Model & Predictions Saved Successfully!")


✅ Model & Predictions Saved Successfully!


In [29]:
predictions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22421 entries, 0 to 22420
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Actual Sales     22421 non-null  float64
 1   Predicted Sales  22421 non-null  float32
dtypes: float32(1), float64(1)
memory usage: 262.9 KB


In [30]:
import joblib

# Save trained AI model
joblib.dump(model, "/content/retail_demand_forecaster.pkl")

print("✅ Model Saved Successfully at /content/")


✅ Model Saved Successfully at /content/
